In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [3]:
df=pd.read_csv('structured_data.csv')
df.head()

,CustomerKey,Customer ID,OrderDateKey,Date,SalesOrderLineKey,Sales Order,Sales Amount
0,-1,[Not Applicable],20170702,7/2/2017,43659001,SO43659,"$2,024.99"
1,-1,[Not Applicable],20170702,7/2/2017,43659002,SO43659,"$6,074.98"
2,-1,[Not Applicable],20170702,7/2/2017,43659003,SO43659,"$2,024.99"
3,-1,[Not Applicable],20170702,7/2/2017,43659004,SO43659,"$2,039.99"
4,-1,[Not Applicable],20170702,7/2/2017,43659005,SO43659,"$2,039.99"


In [4]:
df['Date'] = pd.to_datetime(df['Date'])
datatypes = df['Date'].dtypes
print(datatypes)
datatypes1 = df['CustomerKey'].dtypes
print(datatypes1)
datatypes2 = df['Customer ID'].dtypes
print(datatypes2)
datatypes3 = df['SalesOrderLineKey'].dtypes
print(datatypes3)
datatypes4 = df['Sales Order'].dtypes
print(datatypes4)
datatypes5 = df['Sales Amount'].dtypes
print(datatypes5)


datetime64[ns]
int64
object
int64
object
object


In [5]:
datatypes = pd.to_datetime(df['Date'])
print(datatypes)

0        2017-07-02
1        2017-07-02
2        2017-07-02
3        2017-07-02
4        2017-07-02
            ...    
121248   2019-12-18
121249   2019-12-18
121250   2018-01-13
121251   2019-02-19
121252   2019-02-10
Name: Date, Length: 121253, dtype: datetime64[ns]


In [6]:
df = df[df['CustomerKey'] != -1]
df.head()

,CustomerKey,Customer ID,OrderDateKey,Date,SalesOrderLineKey,Sales Order,Sales Amount
60855,11000,AW00011000,20170712,2017-07-12,43793001,SO43793,"$3,399.99"
60856,11000,AW00011000,20190621,2019-06-21,51522001,SO51522,"$2,319.99"
60857,11000,AW00011000,20190621,2019-06-21,51522002,SO51522,$21.98
60858,11000,AW00011000,20191004,2019-10-04,57418001,SO57418,"$2,384.07"
60859,11000,AW00011000,20191004,2019-10-04,57418002,SO57418,$28.99


In [7]:
df['Sales Amount'] = df['Sales Amount'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df.head()

,CustomerKey,Customer ID,OrderDateKey,Date,SalesOrderLineKey,Sales Order,Sales Amount
60855,11000,AW00011000,20170712,2017-07-12,43793001,SO43793,3399.99
60856,11000,AW00011000,20190621,2019-06-21,51522001,SO51522,2319.99
60857,11000,AW00011000,20190621,2019-06-21,51522002,SO51522,21.98
60858,11000,AW00011000,20191004,2019-10-04,57418001,SO57418,2384.07
60859,11000,AW00011000,20191004,2019-10-04,57418002,SO57418,28.99


In [8]:
most_recent = df['Date'].max()
print(most_recent)
pivot_df = df.groupby('CustomerKey')['Date'].max()
print(pivot_df)
df['MaxDate'] = df['CustomerKey'].map(pivot_df)
df.head()

2020-06-15 00:00:00
CustomerKey
11000   2019-10-04
11001   2020-05-12
11002   2019-07-27
11003   2019-10-11
11004   2019-10-02
           ...    
29479   2019-02-05
29480   2019-12-18
29481   2018-01-13
29482   2019-02-19
29483   2019-02-10
Name: Date, Length: 18484, dtype: datetime64[ns]


,CustomerKey,Customer ID,OrderDateKey,Date,SalesOrderLineKey,Sales Order,Sales Amount,MaxDate
60855,11000,AW00011000,20170712,2017-07-12,43793001,SO43793,3399.99,2019-10-04
60856,11000,AW00011000,20190621,2019-06-21,51522001,SO51522,2319.99,2019-10-04
60857,11000,AW00011000,20190621,2019-06-21,51522002,SO51522,21.98,2019-10-04
60858,11000,AW00011000,20191004,2019-10-04,57418001,SO57418,2384.07,2019-10-04
60859,11000,AW00011000,20191004,2019-10-04,57418002,SO57418,28.99,2019-10-04


**Calculate Recency**

Recency of each CustomerKey is calculated by subtracting 'MaxDate' from 'most_recent'.

In [37]:
Recency = df.groupby(['CustomerKey']).agg({'Date': lambda x: ((most_recent-x.max()).days)})
Recency.head(10)

,Date
CustomerKey,
11000,255
11001,34
11002,324
11003,248
11004,257
11005,256
11006,244
11007,300
11008,317


In [38]:
dfR = pd.DataFrame(Recency)
dfR['Recency_Score'] = pd.qcut(dfR['Date'], q=4, labels=[1, 2, 3, 4])
dfR.head(10)

,Date,Recency_Score
CustomerKey,,
11000,255,4
11001,34,1
11002,324,4
11003,248,3
11004,257,4
11005,256,4
11006,244,3
11007,300,4
11008,317,4


**Calculate Frequency**

Frequency of each CustomerKey is calculated by counting unique value of 'Sales Order'.

In [17]:
Frequency = df.drop_duplicates(subset=['Sales Order']).groupby(['CustomerKey'])[['Sales Order']].count()
Frequency.head(23)

,Sales Order
CustomerKey,
11000,3
11001,3
11002,3
11003,3
11004,3
11005,3
11006,3
11007,3
11008,3


In [29]:
dfF = pd.DataFrame(Frequency)

percentiles = dfF['Sales Order'].quantile([0.25, 0.5, 0.75, 1])
dfF['Sales Order'] = dfF['Sales Order'].astype(int)

dfF['Frequency_Score'] = pd.cut(dfF['Sales Order'], bins=[-float('inf')] + list(percentiles) + [float('inf')], labels=[4, 3, 2, 1], include_lowest=True, duplicates='drop')
dfF.head(30)

,Sales Order,Frequency_Score
CustomerKey,,
11000,3,2
11001,3,2
11002,3,2
11003,3,2
11004,3,2
11005,3,2
11006,3,2
11007,3,2
11008,3,2


In [30]:
mean = dfF['Sales Order'].mean()
median = dfF['Sales Order'].median()
min_date = dfF['Sales Order'].min()
max_date = dfF['Sales Order'].max()
Q1 = dfF['Sales Order'].quantile(0.25)
Q2 = dfF['Sales Order'].quantile(0.5)
Q3 = dfF['Sales Order'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR
print('1 ', mean, '2 ', median, '3 ', min_date, '4 ', max_date, '5 ', Q1, '6 ', Q2, '7 ', Q3, '8 ', IQR, '9 ', lower_limit, '10 ', upper_limit)

1  1.4963752434537978 2  1.0 3  1 4  28 5  1.0 6  1.0 7  2.0 8  1.0 9  -0.5 10  3.5


In [31]:
Q1 = dfF['Sales Order'].quantile(0.25)
Q3 = dfF['Sales Order'].quantile(0.75)
IQR = Q3 - Q1
outliers = dfF[(dfF['Sales Order'] < (Q1 - 1.5 * IQR)) | (dfF['Sales Order'] > (Q3 + 1.5 * IQR))]
outliers.head()

#sns.boxplot(x=dfF['Sales Order'])

,Sales Order,Frequency_Score
CustomerKey,,
11019,17,2
11078,17,2
11091,28,2
11131,5,2
11142,17,2


**Calculate Monetary**

Monentary of each CustomerKey is calculated by summing 'Sales Amount'.

In [32]:
Monetary = df.groupby(['CustomerKey']).agg({'Sales Amount': lambda x: x.sum()})
Monetary.head(15)

,Sales Amount
CustomerKey,
11000,8248.99
11001,6383.88
11002,8114.04
11003,8139.29
11004,8196.01
11005,8121.33
11006,8119.03
11007,8211.00
11008,8106.31


In [33]:
dfM = pd.DataFrame(Monetary)
dfM['Monetary_Score'] = pd.qcut(dfM['Sales Amount'], q=4, labels=[4, 3, 2, 1])
dfM.head(15)

,Sales Amount,Monetary_Score
CustomerKey,,
11000,8248.99,1
11001,6383.88,1
11002,8114.04,1
11003,8139.29,1
11004,8196.01,1
11005,8121.33,1
11006,8119.03,1
11007,8211.00,1
11008,8106.31,1


In [35]:
Q1 = dfM['Sales Amount'].quantile(0.25)
Q3 = dfM['Sales Amount'].quantile(0.75)
IQR = Q3 - Q1
outliers = dfM[(dfM['Sales Amount'] < (Q1 - 1.5 * IQR)) | (dfM['Sales Amount'] > (Q3 + 1.5 * IQR))]
outliers.head()

#sns.boxplot(x=dfM['Sales Amount'])

,Sales Amount,Monetary_Score
CustomerKey,,
11000,8248.99,1
11001,6383.88,1
11002,8114.04,1
11003,8139.29,1
11004,8196.01,1


In [36]:
RFM = pd.concat([dfR, dfF, dfM], axis=1)
dfR.columns = ['Recency', 'Recency_Score']
dfF.columns = ['Frequency', 'Frequency_Score']
dfM.columns = ['Monetary', 'Monetary_Score']
RFM['RFM_Score'] = RFM['Recency_Score'].astype(str) + RFM['Frequency_Score'].astype(str) + RFM['Monetary_Score'].astype(str)
RFM.head(10)

,Recency,Recency_Score,Sales Order,Frequency_Score,Sales Amount,Monetary_Score,RFM_Score
CustomerKey,,,,,,,
11000,255,4,3,2,8248.99,1,421
11001,34,1,3,2,6383.88,1,121
11002,324,4,3,2,8114.04,1,421
11003,248,3,3,2,8139.29,1,321
11004,257,4,3,2,8196.01,1,421
11005,256,4,3,2,8121.33,1,421
11006,244,3,3,2,8119.03,1,321
11007,300,4,3,2,8211.00,1,421
11008,317,4,3,2,8106.31,1,421
